In [1]:
import gdown
import zipfile

# Step 1: Download the zip file from Google Drive
file_id = '1H7OJNxL3fmR1tHqkcUDHRZWq0jhRi-G0'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'dataset.zip'

gdown.download(url, output, quiet=False)

# Step 2: Unzip the dataset
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('dataset')  # You can change 'dataset' to any folder name you prefer


Downloading...
From (original): https://drive.google.com/uc?id=1H7OJNxL3fmR1tHqkcUDHRZWq0jhRi-G0
From (redirected): https://drive.google.com/uc?id=1H7OJNxL3fmR1tHqkcUDHRZWq0jhRi-G0&confirm=t&uuid=c808ad07-195f-4e73-bd1c-fb09e2afcd1a
To: C:\Users\kirti\Desktop\Desktop47\Projects\Green skills Ai internship Edunet\Tree_species_classification\dataset.zip
100%|██████████| 76.3M/76.3M [00:07<00:00, 9.68MB/s]
